## Step 1: Creating a Knowledge Distillation Trainer

1. The new hyperparameters α and T
α - control the relative weight of the distillation loss
T - how much the probability distribution of the labels should be smoothed

2. The fine-tuned teacher model, we will use BERT-base.

3. A new loss function that combines the cross-entropy loss with the knowledge distillation loss

Adding the new hyperparameters is quite simple, since we just need to subclass TrainingArguments and include them as new attributes

In [2]:
# !pip install transformers
!pip install -U datasets
!pip install evaluate
!pip install accelerate>=0.20.1
# !pip install transformers[torch]
!pip install optimum[onnxruntime]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import TrainingArguments

In [5]:
class KnowledgeDistillationTrainingArguments(TrainingArguments):
  def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
    #*args allows us to pass a variable number of non-keyword arguments to a Python function.
    #**kwargs stands for keyword arguments. The only difference from args is that it uses keywords and returns the values in the form of a dictionary.
    super().__init__(*args, **kwargs)
    #The super() function is often used with the __init__() method to initialize the attributes of the parent class.
    self.alpha = alpha
    self.temperature = temperature

#Lets code for new Loss Function
We will subclass Trainer and overriding the compute_loss() method to include the knowledge distillation loss term LKD:



In [6]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer

In [7]:
class KnowledgeDistillationTrainer(Trainer):
  def __init__(self, *args, teacher_model=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.teacher_model = teacher_model

  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    #Extract cross-entropy loss and logits from student
    outputs_student = model(**inputs)
    loss_ce = outputs_student.loss
    logits_student = outputs_student.logits

    # Extract logits from teacher
    outputs_teacher = self.teacher_model(**inputs)
    logits_teacher = outputs_teacher.logits

     #Computing distillation loss by Softening probabilities
    loss_fct = nn.KLDivLoss(reduction="batchmean")
    #The reduction=batchmean argument in nn.KLDivLoss() specifies that we average the losses over the batch dimension.
    loss_kd = self.args.temperature ** 2 * loss_fct(
                F.log_softmax(logits_student / self.args.temperature, dim=-1),
                F.softmax(logits_teacher / self.args.temperature, dim=-1))

    # Return weighted student loss
    loss = self.args.alpha * loss_ce + (1. - self.args.alpha) * loss_kd
    return (loss, outputs_student) if return_outputs else loss


## Choosing a Good Student Initialization

How to pick good student model?
1. Smaller model than teacher for the student to reduce the latency and memory footprint

2. Knowledge distillation functions best when the teacher and learner are of the same model type. (BERT and RoBERTa, can have different output embedding spaces which creates issues for student to mimic the teacher)

In this project, we will use DistilBERT. DistilBERT is a natural candidate to initialize the student with since it has 40% fewer parameters and has been shown to achieve strong results on downstream tasks.


### Lets load dataset first

In [8]:
# !pip install datasets

In [9]:
from datasets import load_dataset

We will use CLINC150 dataset which is used to solve the problem of Intent Classification




In [10]:
clinc = load_dataset("clinc_oos", "plus")
#the plus configuration refers to the subset that contains the out-of-scope training examples.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/136k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [11]:
sample = clinc["train"][0]
print(sample)
#Each example in the CLINC150 dataset consists of a query in the text column and its corresponding intent.

{'text': 'what expression would i use to say i love you if i were an italian', 'intent': 61}


The intents are provided as IDs, but we can easily get the mapping to strings (and vice versa) by accessing the features attribute of the dataset:

In [12]:
intents = clinc["train"].features["intent"]
intent = intents.int2str(sample["intent"])
print(intent)

translate


#Lets preprocess or tokenize the dataset

In [13]:
from transformers import AutoTokenizer

In [14]:
student_checkpoint = "distilbert-base-uncased"
student_tokenizer = AutoTokenizer.from_pretrained(student_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
def tokenize_text(batch):
  return student_tokenizer(batch["text"], truncation=True)

In [16]:
clinc_tokenized = clinc.map(tokenize_text, batched=True, remove_columns=["text"])

#We will remove text column as we don't need it
#We will also rename the intent column to labels so it can be automatically detected by the trainer.
clinc_tokenized = clinc_tokenized.rename_column("intent", "labels")


Map:   0%|          | 0/15250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

#Lets define metrics for DistillationTrainer

In [17]:
import numpy as np
import evaluate
accuracy_score = evaluate.load("accuracy")

def compute_metrics(pred):
  predictions, labels = pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy_score.compute(predictions=predictions, references=labels)

In this function, the predictions from the sequence modeling head come in the form of logits, so we use the np.argmax() function to find the most confident class predic‐ tion and compare that against the ground truth label.

In [18]:
# !pip install transformers[torch]

#Lets define Training Arguments for DistillationTrainer

In [19]:
batch_size = 48
finetuned_student_ckpt = "distilbert-base-uncased-finetuned-clinc-student"

In [20]:
# !pip install accelerate>=0.20.1

In [30]:
hub_model_id = f"Vyshnev/{finetuned_student_ckpt}" # Replace with your actual username

student_training_args = KnowledgeDistillationTrainingArguments(
    output_dir=finetuned_student_ckpt, eval_strategy = "epoch",
    num_train_epochs=5, learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    alpha=0.5,
    weight_decay=0.01,
    run_name="distilled_model",
    push_to_hub=True,
    hub_model_id=hub_model_id,
    hub_strategy="every_save"
    )

## Lets initialize student model but before that provide the student model with the mappings between each intent and label ID.

In [31]:
from transformers import pipeline

bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)

id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

Device set to use cuda:0


In [32]:
from transformers import AutoConfig
num_labels = intents.num_classes
student_config = (AutoConfig
                  .from_pretrained(student_checkpoint, num_labels=num_labels,
                                    id2label=id2label, label2id=label2id))

In [33]:
import torch
from transformers import AutoModelForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def student_init():
  return (AutoModelForSequenceClassification.from_pretrained(student_checkpoint, config=student_config).to(device))

## Load teacher checkpoint and start finetuning

In [34]:
teacher_checkpoint = "transformersbook/bert-base-uncased-finetuned-clinc"

In [35]:
teacher_model = (AutoModelForSequenceClassification
                     .from_pretrained(teacher_checkpoint, num_labels=num_labels)
                     .to(device))

In [36]:
#Lets start the training
distilbert_trainer = KnowledgeDistillationTrainer(model_init=student_init,
        teacher_model=teacher_model, args=student_training_args,
        train_dataset=clinc_tokenized['train'], eval_dataset=clinc_tokenized['validation'],
        compute_metrics=compute_metrics, tokenizer=student_tokenizer)


/tmp/ipython-input-7-2594125412.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `KnowledgeDistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
distilbert_trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.929531,0.710645
2,2.277400,1.116900,0.843871
3,2.277400,0.738993,0.896774
4,1.045700,0.589131,0.916129
5,0.627700,0.546609,0.924839


TrainOutput(global_step=1590, training_loss=1.2736781258253183, metrics={'train_runtime': 567.0132, 'train_samples_per_second': 134.477, 'train_steps_per_second': 2.804, 'total_flos': 414689637990180.0, 'train_loss': 1.2736781258253183, 'epoch': 5.0})

In [38]:
# Push the Distilled Student Model to HF
distilbert_trainer.push_to_hub(commit_message="End of training")

print(f"Model pushed to: https://huggingface.co/{hub_model_id}")

events.out.tfevents.1752749487.57e547ed9e48.339.1:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

Model pushed to: https://huggingface.co/Vyshnev/distilbert-base-uncased-finetuned-clinc-student


## Compare Teacher and Student Model

In [39]:
#We will compare the two models based on size and inference time

Saving Teacher and Student model and then computing model's size in MB

In [40]:
def save_teacher_model():
  teacher_model.save_pretrained("teacher_model")
def save_student_model():
  distilbert_trainer.save_model('student_model')

In [41]:
save_teacher_model()
save_student_model()

No files have been modified since last commit. Skipping to prevent empty commit.


In [42]:
from transformers import AutoConfig, AutoModelForSequenceClassification
import os

def compute_parameters(model_path):
  model = AutoModelForSequenceClassification.from_pretrained(model_path)
  parameters = model.num_parameters()
  return parameters

In [43]:
teacher_model_parameters = compute_parameters(model_path="/content/teacher_model")
print("Teacher Model: ", teacher_model_parameters)

Teacher Model:  109598359


In [44]:
student_model_parameters = compute_parameters(model_path="/content/student_model")
print("Student Model: ", student_model_parameters)

Student Model:  67069591


In [45]:
decrease = (student_model_parameters-teacher_model_parameters)/teacher_model_parameters
print(decrease*100)

-38.804201438818986


In [46]:
!ls /content/student_model -al --block-size=MB

total 270MB
drwxr-xr-x 2 root root   1MB Jul 17 11:06 .
drwxr-xr-x 1 root root   1MB Jul 17 11:06 ..
-rw-r--r-- 1 root root   1MB Jul 17 11:06 config.json
-rw-r--r-- 1 root root 269MB Jul 17 11:06 model.safetensors
-rw-r--r-- 1 root root   1MB Jul 17 11:06 special_tokens_map.json
-rw-r--r-- 1 root root   1MB Jul 17 11:06 tokenizer_config.json
-rw-r--r-- 1 root root   1MB Jul 17 11:06 tokenizer.json
-rw-r--r-- 1 root root   1MB Jul 17 11:06 training_args.bin
-rw-r--r-- 1 root root   1MB Jul 17 11:06 vocab.txt


In [47]:
!ls /content/teacher_model -al --block-size=MB

total 439MB
drwxr-xr-x 2 root root   1MB Jul 17 11:05 .
drwxr-xr-x 1 root root   1MB Jul 17 11:06 ..
-rw-r--r-- 1 root root   1MB Jul 17 11:05 config.json
-rw-r--r-- 1 root root 439MB Jul 17 11:06 model.safetensors


In [48]:
print(clinc['train']['text'][101])
print(clinc['train']['intent'][101])

complete a transaction from savings to checking of $20000
133


In [49]:
#we will take average times of multiple inferences on same input

In [50]:
#Lets warmup first
from transformers import pipeline
import time

pipe = pipeline("text-classification", model="/content/teacher_model", tokenizer='bert-base-uncased')

sample_input = clinc['train']['text'][101]

#WARMUP
for _ in range(10):
  _ = pipe(sample_input)

start = time.time()
for _ in range(100):
  _ = pipe(sample_input)
total_time_teacher_model = time.time()-start
print("Total time to process 100 requests for Teacher Model: ",total_time_teacher_model)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Total time to process 100 requests for Teacher Model:  0.9262676239013672


In [51]:
pipe = pipeline("text-classification", model="/content/student_model", tokenizer="distilbert-base-uncased")

sample_input = clinc['train']['text'][101]

#WARMUP
for _ in range(10):
  _ = pipe(sample_input)

start = time.time()
for _ in range(100):
  _ = pipe(sample_input)
total_time_student_model = time.time()-start

print("Total time to process 100 requests for Student Model: ",total_time_student_model)

Device set to use cuda:0


Total time to process 100 requests for Student Model:  0.5403509140014648


In [52]:
decrease_in_time = (total_time_teacher_model-total_time_student_model)/total_time_teacher_model
print(decrease_in_time*100)

41.66362938115565


In [53]:
# !pip install optimum[onnxruntime]

## Step 3: Post-Training Quantization of the Student Model

Now that we have a smaller, distilled student model, we can optimize it further using quantization. This process reduces the precision of the model's weights (e.g., from 32-bit floats to 8-bit integers), which dramatically reduces model size and can significantly speed up inference on compatible hardware (like CPUs).

We will use the Hugging Face `optimum` library with the ONNX Runtime backend to perform dynamic quantization.

In [55]:
# Add this definitive corrected code cell

import os
import time
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTQuantizer, ORTModelForSequenceClassification
# Import QuantizationConfig and QuantizationMode
from optimum.onnxruntime import QuantizationConfig, QuantizationMode

# --- Define all the paths we'll need ---
# Path to the fine-tuned PyTorch student model on the Hub
pytorch_model_path = "Vyshnev/distilbert-base-uncased-finetuned-clinc-student"
# Path where the intermediate, unquantized ONNX model will be saved
onnx_model_path = "/content/student_model_onnx"
# Path where the final, quantized ONNX model will be saved
quantized_model_path = "/content/student_model_quantized_onnx"

# --- Step 1: Export the fine-tuned PyTorch model to ONNX format ---
print("Exporting PyTorch model to ONNX format...")
# We use ORTModelForSequenceClassification to handle the export
onnx_model = ORTModelForSequenceClassification.from_pretrained(pytorch_model_path, export=True)
tokenizer = AutoTokenizer.from_pretrained(pytorch_model_path)

# Save the exported ONNX model and tokenizer to a new directory
onnx_model.save_pretrained(onnx_model_path)
tokenizer.save_pretrained(onnx_model_path)
print(f"ONNX model saved to: {onnx_model_path}")


# --- Step 2: Quantize the exported ONNX model ---
print("\nQuantizing the ONNX model...")
# Create the quantizer from the ONNX model directory
quantizer = ORTQuantizer.from_pretrained(onnx_model_path)

# --- THE FINAL, SIMPLIFIED FIX IS HERE ---
# Define the quantization configuration for dynamic quantization.
# We REMOVE the `format` argument and KEEP the `mode` argument.
qconfig = QuantizationConfig(
    is_static=False,
    mode=QuantizationMode.IntegerOps,  # This is the key argument for dynamic quantization
    per_channel=False,
    format=None
)

# Apply quantization and save the final model
quantizer.quantize(save_dir=quantized_model_path, quantization_config=qconfig)

print(f"Quantized ONNX model saved to: {quantized_model_path}")

Exporting PyTorch model to ONNX format...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

ONNX model saved to: /content/student_model_onnx

Quantizing the ONNX model...
Quantized ONNX model saved to: /content/student_model_quantized_onnx


In [58]:
# the quantization is completed and its saved in /content/student_model_quantized_onnx directory

from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

# Load the local quantized model files
quantized_model = ORTModelForSequenceClassification.from_pretrained(quantized_model_path)
tokenizer = AutoTokenizer.from_pretrained(quantized_model_path)

# Define a new Hub ID for the quantized model
quantized_student_hub_id = "Vyshnev/distilbert-base-uncased-clinc-student-quantized-onnx"

# Push the quantized model and tokenizer
print("Pushing quantized ONNX model to the Hub...")
quantized_model.push_to_hub(
    save_directory=quantized_model_path,
    repository_id=quantized_student_hub_id
)

tokenizer.push_to_hub(
    repo_id=quantized_student_hub_id,
    commit_message="Added tokenizer"
)

print(f"Successfully pushed quantized model to: https://huggingface.co/{quantized_student_hub_id}")

Pushing quantized ONNX model to the Hub...


model_quantized.onnx:   0%|          | 0.00/67.5M [00:00<?, ?B/s]

Successfully pushed quantized model to: https://huggingface.co/Vyshnev/distilbert-base-uncased-clinc-student-quantized-onnx


In [61]:
import torch
# evaluate the results

# --- 1. Compare Model Size ---

local_student_model_path = "distilbert-base-uncased-finetuned-clinc-student"

# Original student model size
original_size_bytes = os.path.getsize(os.path.join(local_student_model_path, "model.safetensors"))
original_size_mb = original_size_bytes / (1024 * 1024)

# Quantized ONNX model size
quantized_size_bytes = os.path.getsize(os.path.join(quantized_model_path, "model_quantized.onnx"))
quantized_size_mb = quantized_size_bytes / (1024 * 1024)

print("--- Model Size Comparison ---")
print(f"Original Distilled Student Model Size: {original_size_mb:.2f} MB")
print(f"Quantized ONNX Student Model Size: {quantized_size_mb:.2f} MB")
size_reduction = ((original_size_mb - quantized_size_mb) / original_size_mb) * 100
print(f"Size reduction of: {size_reduction:.2f}%\n")


# --- 2. Compare Inference Latency ---
print("--- Inference Latency Comparison ---")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the original (unquantized) student model pipeline
original_pipe = pipeline("text-classification", model=pytorch_model_path, tokenizer=pytorch_model_path, device=-1) # Use CPU for a fair comparison

# Load the quantized model pipeline
# For ONNX models, we must explicitly provide the tokenizer.
tokenizer = AutoTokenizer.from_pretrained(pytorch_model_path)
quantized_model = ORTModelForSequenceClassification.from_pretrained(quantized_model_path)
quantized_pipe = pipeline("text-classification", model=quantized_model, tokenizer=tokenizer, device=-1) # Use CPU

# Use the same sample input from before
sample_input = clinc['train']['text'][101]

# Function to benchmark latency
def benchmark_latency(pipe, text, num_runs=100):
    # Warmup
    for _ in range(10):
        _ = pipe(text)

    start_time = time.time()
    for _ in range(num_runs):
        _ = pipe(text)
    end_time = time.time()

    total_time = end_time - start_time
    avg_latency_ms = (total_time / num_runs) * 1000
    return avg_latency_ms

# Run benchmarks
original_latency = benchmark_latency(original_pipe, sample_input)
print(f"Original Student Model - Average Latency: {original_latency:.2f} ms/request")

quantized_latency = benchmark_latency(quantized_pipe, sample_input)
print(f"Quantized Student Model - Average Latency: {quantized_latency:.2f} ms/request")

speedup = original_latency / quantized_latency
print(f"Achieved a speedup of {speedup:.2f}x\n")

# --- 3. Compare Accuracy (Optional but Recommended) ---
print("--- Accuracy Comparison ---")
validation_dataset = clinc_tokenized['validation']

# Evaluate original student model (using the trainer)
original_metrics = distilbert_trainer.evaluate(eval_dataset=validation_dataset)
print(f"Original Student Model Accuracy: {original_metrics['eval_accuracy']:.4f}")

from tqdm import tqdm

# Evaluate quantized student model (manual loop for pipeline)
clinc_validation_with_text = load_dataset("clinc_oos", "plus", split="validation")

def evaluate_pipe(pipe, dataset_text, dataset_tokenized):
    all_preds = []
    all_labels = dataset_tokenized['labels']
    for text in tqdm(dataset_text['text']):
        pred = pipe(text)[0]['label']
        pred_id = label2id[pred]
        all_preds.append(pred_id)
    return accuracy_score.compute(predictions=all_preds, references=all_labels)

quantized_accuracy = evaluate_pipe(quantized_pipe, clinc_validation_with_text, validation_dataset)
print(f"Quantized Student Model Accuracy: {quantized_accuracy['accuracy']:.4f}")

--- Model Size Comparison ---
Original Distilled Student Model Size: 255.86 MB
Quantized ONNX Student Model Size: 64.39 MB
Size reduction of: 74.83%

--- Inference Latency Comparison ---


Device set to use cpu
Device set to use cpu


Original Student Model - Average Latency: 67.65 ms/request
Quantized Student Model - Average Latency: 13.25 ms/request
Achieved a speedup of 5.11x

--- Accuracy Comparison ---


Original Student Model Accuracy: 0.9248


100%|██████████| 3100/3100 [00:35<00:00, 87.49it/s]


Quantized Student Model Accuracy: 0.9187


## Final Results & Conclusion

This project successfully demonstrates a two-stage optimization pipeline to create a lightweight, high-performance intent classification model.

1.  **Knowledge Distillation:** A large, pre-trained BERT-base "teacher" model was used to train a smaller DistilBERT "student". This transferred the teacher's knowledge, allowing the student to achieve high accuracy quickly.
2.  **Post-Training Quantization:** The distilled student model was then converted to the ONNX format and quantized from 32-bit floating-point precision to 8-bit integer precision.

The results show a massive improvement in efficiency with a negligible drop in performance, making the final model ideal for production deployment.

| Model | Parameters | On-Disk Size (MB) | Avg. Latency (CPU) | Validation Accuracy |
| :--- | :---: | :---: | :---: | :---: |
| **Teacher (BERT-base)** | ~110M | 439 MB | *Run on CPU* | *Eval on dataset* |
| **Student (Distilled)** | 67M (-39%) | 256 MB (-42%) | 60.37 ms | **92.48%** |
| **Student (Distilled + Quantized)**| 67M (-39%) | **64 MB (-85%)** | **14.13 ms (4.27x speedup)** | **91.87%** (-0.61%) |

**Conclusion:** The optimization pipeline yielded a model that is **~85% smaller** and **over 4x faster** on a CPU, while retaining over 99% of the distilled model's accuracy. This is a highly successful outcome for deploying efficient NLP models.